<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-Tabbed-Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# IFC Tabbed Dashboard

This notebook creates an interactive dashboard with [Dash](https://dash.plotly.com/) inside Jupyter. It demonstrates how to analyze IFC data and visualize the results in an interative way.

**What you'll learn:**
- Load IFC files from GitHub or upload your own
- Extract building elements, storeys, and materials
- Visualize data in an interactive tabbed dashboard
- Identify unassigned elements and data quality issues


## 1. Install Dependencies

Run this cell to install required Python packages. This is needed in Colab or fresh environments.


In [ ]:
%pip install ifcopenshell pandas plotly dash ipywidgets


## 2. Load IFC Model

Choose how to load your IFC model:
- **Option A**: Download from GitHub repository (works in Colab)
- **Option B**: Upload your own IFC file

The code extracts:
- Building storeys (from `IfcRelContainedInSpatialStructure`)
- Element types (e.g., `IfcWall`, `IfcDoor`, `IfcWindow`)
- Materials (from `IfcRelAssociatesMaterial`)

Result: A clean DataFrame with one row per element-material combination.


In [ ]:
from __future__ import annotations

from collections import defaultdict
from pathlib import Path
import os
import warnings

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Filter FutureWarnings from seaborn/pandas
warnings.filterwarnings('ignore', category=FutureWarning)

try:
    import ifcopenshell
except ImportError as exc:
    raise ImportError("ifcopenshell is required for this notebook. Install it via `pip install ifcopenshell`.") from exc

# Global variables for storing the loaded model and data
model = None
ifc_df = None
storey_by_element = {}
materials_by_element = {}

def storey_label(storey):
    """Return a readable label for a building storey."""
    if storey is None:
        return 'Not assigned'
    return storey.LongName or storey.Name or f"Storey {storey.id()}"


def material_names(material) -> list[str]:
    """Return a list of material names for different IFC material types."""
    if material is None:
        return ['Unspecified']
    if material.is_a('IfcMaterial'):
        return [material.Name or 'Unnamed material']
    if material.is_a('IfcMaterialList'):
        return [mat.Name or 'Unnamed material' for mat in material.Materials]
    if material.is_a('IfcMaterialLayerSetUsage'):
        return material_names(material.ForLayerSet)
    if material.is_a('IfcMaterialLayerSet'):
        names = []
        for layer in material.MaterialLayers:
            if layer.Material:
                names.append(layer.Material.Name or 'Unnamed material')
        return names or ['Unspecified']
    if material.is_a('IfcMaterialConstituentSet'):
        names = []
        for constituent in material.MaterialConstituents:
            if constituent.Material:
                names.append(constituent.Material.Name or 'Unnamed material')
        return names or ['Unspecified']
    if material.is_a('IfcMaterialProfileSetUsage'):
        return material_names(material.ForProfileSet)
    if material.is_a('IfcMaterialProfileSet'):
        names = []
        for profile in material.MaterialProfiles:
            if profile.Material:
                names.append(profile.Material.Name or 'Unnamed material')
        return names or ['Unspecified']
    fallback = getattr(material, "Name", None)
    return [fallback or 'Unspecified']


def load_ifc_model(file_path):
    """Load and process IFC model from the given path."""
    global model, ifc_df, storey_by_element, materials_by_element
    
    try:
        # Load the IFC model
        model = ifcopenshell.open(file_path)
        
        # Clear and reinitialize previous data
        storey_by_element = {}
        materials_by_element = defaultdict(list)
        
        # First pass: Map elements directly contained in storeys
        for relation in model.by_type('IfcRelContainedInSpatialStructure'):
            structure = relation.RelatingStructure
            if structure and structure.is_a('IfcBuildingStorey'):
                label = storey_label(structure)
                for element in relation.RelatedElements:
                    storey_by_element[element.GlobalId] = label
        
        # Second pass: Check for elements in spaces that belong to storeys
        for relation in model.by_type('IfcRelContainedInSpatialStructure'):
            structure = relation.RelatingStructure
            if structure and structure.is_a('IfcSpace'):
                # Find which storey this space belongs to
                space_storey = None
                for space_relation in model.by_type('IfcRelContainedInSpatialStructure'):
                    if structure in space_relation.RelatedElements:
                        parent = space_relation.RelatingStructure
                        if parent and parent.is_a('IfcBuildingStorey'):
                            space_storey = storey_label(parent)
                            break
                
                # Assign elements in this space to the space's storey
                if space_storey:
                    for element in relation.RelatedElements:
                        if element.GlobalId not in storey_by_element:
                            storey_by_element[element.GlobalId] = f"{space_storey} (via Space)"

        # Map each element (by GlobalId) to the materials applied to it
        for relation in model.by_type('IfcRelAssociatesMaterial'):
            names = material_names(relation.RelatingMaterial)
            for element in relation.RelatedObjects:
                materials_by_element[element.GlobalId].extend(names)

        # Remove duplicated material names per element
        for element_id, names in materials_by_element.items():
            deduplicated: list[str] = []
            for name in names:
                if name not in deduplicated:
                    deduplicated.append(name)
            materials_by_element[element_id] = deduplicated

        # Create records DataFrame
        records: list[dict[str, str]] = []
        unassigned_count = 0
        
        for element in model.by_type('IfcElement'):
            element_id = element.GlobalId
            element_type = element.is_a()
            storey = storey_by_element.get(element_id, 'Not assigned')
            
            if storey == 'Not assigned':
                unassigned_count += 1
            
            element_materials = materials_by_element.get(element_id, ['Unspecified'])
            for material in element_materials:
                records.append({
                    'ElementId': element_id,
                    'ElementType': element_type,
                    'Storey': storey,
                    'Material': material or 'Unspecified',
                })

        if not records:
            raise ValueError('No element records found. Ensure the IFC model contains elements and materials.')

        ifc_df = pd.DataFrame(records)
        
        # Display success message
        clear_output(wait=True)
        display(loading_widget)
        print(f"✅ IFC model loaded successfully!")
        print(f"📁 File: {Path(file_path).name}")
        print(f"🏗️ Schema: {model.schema}")
        print(f"📊 Elements: {len(model.by_type('IfcElement')):,}")
        print(f"📋 Records: {len(ifc_df):,}")
        print(f"🏢 Storeys: {len(ifc_df['Storey'].unique())}")
        print(f"🧱 Materials: {len(ifc_df['Material'].unique())}")
        
        if unassigned_count > 0:
            print(f"⚠️  Unassigned: {unassigned_count} elements not in any storey")
        
        print("\n📊 First 5 records:")
        display(ifc_df.head())
        
    except Exception as e:
        clear_output(wait=True)
        display(loading_widget)
        print(f"❌ Error loading IFC model: {str(e)}")
        print("Please check the file path and try again.")


def on_load_button_clicked(b):
    """Handle the Load Model button click."""
    if loading_method.value == 'A':
        # Option A: Download from GitHub repository
        if github_files_dropdown.value and github_files_dropdown.value != 'Error fetching from GitHub':
            import urllib.request
            
            selected_file = github_files_dropdown.value
            github_url = f'https://raw.githubusercontent.com/{GITHUB_REPO_OWNER}/{GITHUB_REPO_NAME}/main/{GITHUB_FOLDER_PATH}/{selected_file}'
            
            print(f"📥 Downloading {selected_file} from GitHub...")
            try:
                # Download file to temporary location
                local_filename = selected_file
                urllib.request.urlretrieve(github_url, local_filename)
                print(f"✅ Downloaded successfully!")
                
                load_ifc_model(local_filename)
            except Exception as e:
                print(f"❌ Error downloading file: {e}")
        else:
            print("❌ Please select a valid file from the dropdown.")
    
    elif loading_method.value == 'B':
        # Option B: File upload widget (works in Jupyter/Colab)
        if file_upload.value:
            uploaded_file = next(iter(file_upload.value.values()))
            file_content = uploaded_file['content']
            file_name = uploaded_file['metadata']['name']
            
            # Save uploaded file temporarily
            with open(file_name, 'wb') as f:
                f.write(file_content)
            
            load_ifc_model(file_name)
        else:
            print("❌ Please upload an IFC file.")


# Create the interactive loading interface
print("🔧 IFC Model Loading Options")
print("Choose how you want to load your IFC model:")

# Option selection
loading_method = widgets.RadioButtons(
    options=[
        ('A) Download from GitHub Repository', 'A'),
        ('B) Upload File from Your Computer', 'B')
    ],
    value='A',
    description='Method:',
    style={'description_width': 'initial'}
)

# Option A: GitHub repository files
# GitHub repository details
GITHUB_REPO_OWNER = 'louistrue'
GITHUB_REPO_NAME = 'learn-ifc-bfh25-D'
GITHUB_FOLDER_PATH = 'Modelle/BFH-25'

def fetch_github_files():
    """Fetch list of IFC files from GitHub repository."""
    import urllib.request
    import json
    
    url = f'https://api.github.com/repos/{GITHUB_REPO_OWNER}/{GITHUB_REPO_NAME}/contents/{GITHUB_FOLDER_PATH}'
    try:
        with urllib.request.urlopen(url) as response:
            files_data = json.loads(response.read().decode())
            ifc_files = [f['name'] for f in files_data if f['name'].endswith('.ifc')]
            return ifc_files
    except Exception as e:
        print(f"⚠️ Could not fetch files from GitHub: {e}")
        return ['Error fetching from GitHub']

github_file_options = fetch_github_files()

github_files_dropdown = widgets.Dropdown(
    options=github_file_options,
    value=github_file_options[0] if github_file_options else None,
    description='GitHub File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Option B: File upload widget
file_upload = widgets.FileUpload(
    accept='.ifc',
    multiple=False,
    description='Upload IFC:',
    style={'description_width': 'initial'}
)

# Load button
load_button = widgets.Button(
    description='Load Model',
    button_style='success',
    icon='download',
    layout=widgets.Layout(width='150px')
)
load_button.on_click(on_load_button_clicked)

# Create the main widget container
loading_widget = widgets.VBox([
    loading_method,
    widgets.HBox([github_files_dropdown]),
    widgets.HBox([file_upload]),
    widgets.HBox([load_button])
])

# Display the loading interface
display(loading_widget)

# Auto-load default file if available from GitHub
if github_file_options and github_file_options[0] != 'Error fetching from GitHub':
    print("🔄 Auto-downloading and loading default file from GitHub...")
    import urllib.request
    
    selected_file = github_file_options[0]
    github_url = f'https://raw.githubusercontent.com/{GITHUB_REPO_OWNER}/{GITHUB_REPO_NAME}/main/{GITHUB_FOLDER_PATH}/{selected_file}'
    
    try:
        local_filename = selected_file
        urllib.request.urlretrieve(github_url, local_filename)
        load_ifc_model(local_filename)
    except Exception as e:
        print(f"⚠️ Auto-load failed: {e}")
        print("📁 Please select a loading method and click 'Load Model' to continue.")
else:
    print("📁 Please select a loading method and click 'Load Model' to continue.")


## 3. Prepare Data for Visualization

Create aggregated DataFrames for the dashboard:
- **storey_summary**: Element counts per storey
- **storey_element_breakdown**: Element types per storey
- **storey_material_breakdown**: Materials per storey
- **material_element_breakdown**: Material-element combinations
- **unassigned_by_type**: Elements without storey assignment


In [ ]:
storey_summary = (
    ifc_df.groupby('Storey')['ElementId']
    .nunique()
    .reset_index(name='ElementCount')
    .sort_values('ElementCount', ascending=False)
)

storey_element_breakdown = (
    ifc_df.groupby(['Storey', 'ElementType'])['ElementId']
    .nunique()
    .reset_index(name='ElementCount')
    .sort_values(['Storey', 'ElementCount'], ascending=[True, False])
)

storey_material_breakdown = (
    ifc_df.groupby(['Storey', 'Material'])['ElementId']
    .nunique()
    .reset_index(name='ElementCount')
    .sort_values(['Storey', 'ElementCount'], ascending=[True, False])
)

material_element_breakdown = (
    ifc_df.groupby(['Material', 'ElementType'])['ElementId']
    .nunique()
    .reset_index(name='ElementCount')
    .sort_values('ElementCount', ascending=False)
)

# Unassigned elements analysis
unassigned_elements = ifc_df[ifc_df['Storey'] == 'Not assigned'].copy()

unassigned_by_type = (
    unassigned_elements.groupby('ElementType')['ElementId']
    .nunique()
    .reset_index(name='Count')
    .sort_values('Count', ascending=False)
)

storey_summary.head()


## 4. Build Interactive Dashboard

Creates a Dash app with multiple tabs:
- **Storey Overview**: Element counts and type distribution
- **Material Insights**: Material usage and combinations
- **Unassigned Elements**: Quality control for spatial organization

The dashboard runs directly in the notebook using Plotly for interactive charts.


In [ ]:
from dash import Dash, dash_table, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

# Function to create figures with optional filtering
def create_figures(include_unassigned=True):
    """Create all dashboard figures with optional filtering of unassigned elements."""
    
    # Filter data if needed
    if include_unassigned:
        filtered_df = ifc_df
    else:
        filtered_df = ifc_df[ifc_df['Storey'] != 'Not assigned']
    
    # Recreate aggregations with filtered data
    filtered_storey_summary = (
        filtered_df.groupby('Storey')['ElementId']
        .nunique()
        .reset_index(name='ElementCount')
        .sort_values('ElementCount', ascending=False)
    )
    
    filtered_storey_element_breakdown = (
        filtered_df.groupby(['Storey', 'ElementType'])['ElementId']
        .nunique()
        .reset_index(name='ElementCount')
        .sort_values(['Storey', 'ElementCount'], ascending=[True, False])
    )
    
    filtered_storey_material_breakdown = (
        filtered_df.groupby(['Storey', 'Material'])['ElementId']
        .nunique()
        .reset_index(name='ElementCount')
        .sort_values(['Storey', 'ElementCount'], ascending=[True, False])
    )
    
    filtered_material_element_breakdown = (
        filtered_df.groupby(['Material', 'ElementType'])['ElementId']
        .nunique()
        .reset_index(name='ElementCount')
        .sort_values('ElementCount', ascending=False)
    )
    
    # Create figures
    storey_total_fig = px.bar(
        filtered_storey_summary,
        x='Storey',
        y='ElementCount',
        text_auto='.0f',
        title='Elements per building storey',
        color='ElementCount',
        color_continuous_scale='Blues'
    )
    storey_total_fig.update_layout(coloraxis_showscale=False, xaxis_title='', yaxis_title='Number of elements')
    
    storey_type_fig = px.bar(
        filtered_storey_element_breakdown,
        x='Storey',
        y='ElementCount',
        color='ElementType',
        title='Element type distribution per storey',
        text_auto='.0f'
    )
    storey_type_fig.update_layout(xaxis_title='', yaxis_title='Number of elements', legend_title='Element type')
    
    material_storey_fig = px.bar(
        filtered_storey_material_breakdown,
        x='Storey',
        y='ElementCount',
        color='Material',
        title='Material usage per storey',
        text_auto='.0f'
    )
    material_storey_fig.update_layout(xaxis_title='', yaxis_title='Elements using material', legend_title='Material')
    
    material_heatmap_fig = px.density_heatmap(
        filtered_material_element_breakdown,
        x='ElementType',
        y='Material',
        z='ElementCount',
        title='Material vs element type',
        color_continuous_scale='Viridis'
    )
    material_heatmap_fig.update_layout(xaxis_title='Element type', yaxis_title='Material')
    
    top_material_combinations = filtered_material_element_breakdown.head(15)
    
    return {
        'storey_total': storey_total_fig,
        'storey_type': storey_type_fig,
        'material_storey': material_storey_fig,
        'material_heatmap': material_heatmap_fig,
        'top_materials': top_material_combinations
    }

# Initial figures
initial_figures = create_figures(include_unassigned=True)

# Unassigned elements visualization
if len(unassigned_by_type) > 0:
    unassigned_fig = px.bar(
        unassigned_by_type,
        x='ElementType',
        y='Count',
        text_auto='.0f',
        title=f'Unassigned Elements by Type ({len(unassigned_elements["ElementId"].unique())} total)',
        color='Count',
        color_continuous_scale='Reds'
    )
    unassigned_fig.update_layout(
        coloraxis_showscale=False, 
        xaxis_title='Element Type', 
        yaxis_title='Count',
        xaxis_tickangle=-45
    )
else:
    unassigned_fig = None

# Create tab list
tabs_list = [
    dcc.Tab(
        label='Storey overview',
        children=[
            html.H3('How many elements exist per storey?'),
            html.P('Counts are based on unique element GlobalIds assigned to each building storey.'),
            dcc.Graph(id='storey-total-graph', figure=initial_figures['storey_total']),
            html.Hr(),
            html.H3('Which element types dominate each storey?'),
            html.P('Stacked bars highlight the element categories present on each storey.'),
            dcc.Graph(id='storey-type-graph', figure=initial_figures['storey_type']),
        ],
    ),
    dcc.Tab(
        label='Material insights',
        children=[
            html.H3('Which materials are used on each storey?'),
            html.P('Materials are derived from IfcRelAssociatesMaterial and summarised per storey.'),
            dcc.Graph(id='material-storey-graph', figure=initial_figures['material_storey']),
            html.Hr(),
            html.H3('Material vs element type'),
            html.P('Use the heatmap to spot dominant material-element combinations.'),
            dcc.Graph(id='material-heatmap-graph', figure=initial_figures['material_heatmap']),
            html.Hr(),
            html.H3('Top material & element combinations'),
            html.Div(id='material-table-container', children=[
                dash_table.DataTable(
                    data=initial_figures['top_materials'].to_dict('records'),
                    columns=[{'name': c, 'id': c} for c in initial_figures['top_materials'].columns],
                    style_table={'maxHeight': '400px', 'overflowY': 'auto'},
                    style_cell={'padding': '0.5rem', 'textAlign': 'left'},
                    style_header={'fontWeight': 'bold'},
                )
            ]),
        ],
    ),
]

# Add unassigned elements tab if there are any
if len(unassigned_by_type) > 0:
    tabs_list.append(
        dcc.Tab(
            label=f'⚠️ Unassigned ({len(unassigned_elements["ElementId"].unique())})',
            children=[
                html.H3('Elements Not Assigned to Any Storey'),
                html.P([
                    'These elements are not directly contained in an ',
                    html.Code('IfcBuildingStorey'),
                    ' or indirectly through an ',
                    html.Code('IfcSpace'),
                    '. They may need to be reviewed in the source model.'
                ]),
                dcc.Graph(figure=unassigned_fig),
                html.Hr(),
                html.H3('Unassigned Elements Details'),
                dash_table.DataTable(
                    data=unassigned_by_type.to_dict('records'),
                    columns=[{'name': c, 'id': c} for c in unassigned_by_type.columns],
                    style_table={'maxHeight': '400px', 'overflowY': 'auto'},
                    style_cell={'padding': '0.5rem', 'textAlign': 'left'},
                    style_header={'fontWeight': 'bold'},
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},
                            'backgroundColor': 'rgb(248, 248, 248)'
                        }
                    ],
                ),
            ],
        )
    )

app.layout = html.Div(
    [
        html.H1('IFC Elements and Materials Explorer'),
        html.P(
            'Use the tabs to explore how elements and materials are distributed across the building. '
            'This starter dashboard can be adapted to any IFC dataset by editing the data preparation cell above.'
        ),
        
        # Toggle for unassigned elements
        html.Div([
            dcc.Checklist(
                id='include-unassigned-toggle',
                options=[
                    {'label': ' Include unassigned elements in visualizations', 'value': 'include'}
                ],
                value=['include'],  # Default: include unassigned
                style={'margin': '10px 0'}
            ),
        ], style={'background': '#f0f0f0', 'padding': '10px', 'border-radius': '5px', 'margin-bottom': '20px'}),
        
        dcc.Tabs(tabs_list),
    ]
)

# Callbacks to update graphs based on toggle
@app.callback(
    [
        Output('storey-total-graph', 'figure'),
        Output('storey-type-graph', 'figure'),
        Output('material-storey-graph', 'figure'),
        Output('material-heatmap-graph', 'figure'),
        Output('material-table-container', 'children')
    ],
    Input('include-unassigned-toggle', 'value')
)
def update_graphs(toggle_value):
    """Update all graphs based on the toggle state."""
    include_unassigned = 'include' in toggle_value
    figures = create_figures(include_unassigned=include_unassigned)
    
    # Create new table
    table = dash_table.DataTable(
        data=figures['top_materials'].to_dict('records'),
        columns=[{'name': c, 'id': c} for c in figures['top_materials'].columns],
        style_table={'maxHeight': '400px', 'overflowY': 'auto'},
        style_cell={'padding': '0.5rem', 'textAlign': 'left'},
        style_header={'fontWeight': 'bold'},
    )
    
    return (
        figures['storey_total'],
        figures['storey_type'],
        figures['material_storey'],
        figures['material_heatmap'],
        table
    )

# Run app in Jupyter
app.run(jupyter_mode='inline', height=900, port=8051)


## 5. Next Steps

**Extend the dashboard:**
- Add dropdown filters for specific storeys or materials
- Export data to CSV for reporting
- Update some data (eg. calculate cost or emissions)

**Try different models:**
- Upload your own IFC files using Option B
- Compare different buildings or design versions
- Analyze data quality and completeness
